# Import Libraries

In [4]:
import numpy as np
import pandas as pd
import random
import math

# Constants Initialization

In [5]:
ALPHA = 4

# affinity weights
WEIGHTS = {
    'CPU' : 0.4,
    'IO' : 0.2,
    'NW' : 0.2,
    'Memory' : 0.2
}

# genetic algorithm
NUMBER_OF_GA_ITERS = 50
POPULATION_SIZE = 20
NUM_NEW_POPULATION = 10
MUTATION_PROB = 0.75

# simulated annealing
NUMBER_OF_SA_ITERS = 5000
STEP_SIZE = 5
TEMP = 100
NUMBER_OF_NEW_CAND_ITERS = 1000

# Load Dataset

In [6]:
df_machines = pd.read_csv('../Datasets/Real-life Dataset/Machines_250.csv')
df_applications = pd.read_csv('../Datasets/Real-life Dataset/Applications_200.csv')
df_anti_affinity_matrix = pd.read_csv('../Datasets/Real-life Dataset/Anti_Affinity_Matrix_200_250.csv')
df_user_affinity_matrix = pd.read_csv('../Datasets/Real-life Dataset/Affinity_Matrix_200_250.csv')

# df_machines = pd.read_csv('../Datasets/Synthetic Dataset/Machines.csv')
# df_applications = pd.read_csv('../Datasets/Synthetic Dataset/Applications.csv')
# df_anti_affinity_matrix = pd.read_csv('../Datasets/Synthetic Dataset/Anti_Affinity_Matrix.csv')
# df_user_affinity_matrix = pd.read_csv('../Datasets/Synthetic Dataset/Affinity_Matrix.csv')

df_machines = df_machines.loc[:, ~df_machines.columns.str.contains('^Unnamed')]
df_applications = df_applications.loc[:, ~df_applications.columns.str.contains('^Unnamed')]
df_anti_affinity_matrix = df_anti_affinity_matrix.loc[:, ~df_anti_affinity_matrix.columns.str.contains('^Unnamed')]
df_user_affinity_matrix = df_user_affinity_matrix.loc[:, ~df_user_affinity_matrix.columns.str.contains('^Unnamed')]

df_anti_affinity_matrix.columns = pd.to_numeric(df_anti_affinity_matrix.columns, errors='coerce')
df_user_affinity_matrix.columns = pd.to_numeric(df_user_affinity_matrix.columns, errors='coerce')

df_machines['Index'] = df_machines.index
df_applications['Index'] = df_applications.index

# Calculate Affinity Matrix

In [7]:
affinity = np.zeros((df_applications.shape[0], df_machines.shape[0]))
for i, application in df_applications.iterrows():
    for j, machine in df_machines.iterrows():
        for col in ['CPU','IO','NW','Memory']:
            if (application[col] > machine[col]):
                affinity[i][j] = 0
                break;
            else:
                affinity[i][j] +=  WEIGHTS[col] * (machine[col] - application[col]) / machine[col]
df_affinity_matrix = (affinity + df_user_affinity_matrix) / 2

# General Helper Functions

In [8]:
def calculate_utilization(df_allocation_matrix, df_applications, df_machines):
    df_allocation_copy = df_allocation_matrix.copy()
    for i, application in df_applications.iterrows():
        df_allocation_copy.loc[i] = df_allocation_copy.loc[i] * application['CPU']
    return df_allocation_copy.sum() / df_machines['CPU']

In [9]:
def calculate_total_power_cost(df_allocation_matrix, df_applications, df_machines):
    utilization = calculate_utilization(df_allocation_matrix, df_applications, df_machines)
    power_consumed = df_machines['P_idle'] + (df_machines['P_max'] - df_machines['P_idle']) * (utilization ** 3)
    return power_consumed.sum()

In [10]:
def calculate_total_affinity_cost(df_allocation_matrix, df_affinity_matrix):
    affinity_cost = df_allocation_matrix * df_affinity_matrix
    return affinity_cost.sum().sum()

In [11]:
def calculate_total_system_cost(power_cost, affinity_cost):
    total_cost = power_cost - ALPHA * affinity_cost
    return total_cost

In [12]:
def calculate_number_of_unused_machines(df_allocation_matrix):
    col_sums = df_allocation_matrix.sum()
    zero_count = (col_sums == 0).sum()
    return zero_count

In [13]:
def calculate_average_affinity_satisfaction_ratio(df_applications, df_machines, df_allocation_matrix, df_user_affinity_matrix):
    result = 0
    for i in range(df_applications.shape[0]):
        for j in range(df_machines.shape[0]):
            result = result + df_allocation_matrix.loc[i,j] * df_user_affinity_matrix.loc[i,j]
    
    return result / df_applications['Instances'].sum()

In [14]:
def calculate_average_utilization(df_allocation_matrix, df_applications, df_machines):
    utilization = calculate_utilization(df_allocation_matrix, df_applications, df_machines)
    return utilization.sum() / df_machines.shape[0]

# Genetic Algorithm

In [15]:
def find_num(machine, application):
    res = []
    for col in ['CPU','IO','NW','Memory']:
        res.append(int(machine[col]/application[col]));
    return min(res);

In [16]:
def random_ffa(df_machines,df_applications,df_affinity_matrix,df_anti_affinity_matrix):
    df_remains = df_machines.copy()
    df_allocation_matrix = pd.DataFrame(np.zeros((df_applications.shape[0], df_machines.shape[0]), dtype=int))
    random_machine_index_list = random.sample(range(df_machines.shape[0]), df_machines.shape[0])
    for index_application, row_application in df_applications.iterrows():
        instances_remaining = row_application['Instances']
        for index_machine in random_machine_index_list:
            if(df_anti_affinity_matrix.loc[index_application,index_machine] == 1):
                continue;
            instances_on_this_machine = find_num(df_remains.loc[index_machine],row_application);
            instances_on_this_machine = min(instances_on_this_machine,instances_remaining)
            df_allocation_matrix.loc[index_application,index_machine] = instances_on_this_machine
            instances_remaining -= instances_on_this_machine
            for col in ['CPU','IO','NW','Memory']:
                df_remains.loc[index_machine,col] = df_remains.loc[index_machine,col] - instances_on_this_machine * row_application[col]
    return df_allocation_matrix;

In [17]:
def generate_suitable_candidate(df_machines,df_applications,df_affinity_matrix,df_anti_affinity_matrix):
    instances_required = df_applications['Instances'].sum()
    while(True):
        df_soln = random_ffa(df_machines,df_applications,df_affinity_matrix,df_anti_affinity_matrix)
        instances_scheduled = df_soln.sum().sum()
        if(instances_scheduled == instances_required):
            return df_soln

In [18]:
def generate_population(df_machines,df_applications,df_affinity_matrix,df_anti_affinity_matrix):
    solution_list = []
    for i in range(POPULATION_SIZE):
        one_soln = generate_suitable_candidate(df_machines,df_applications,df_affinity_matrix,df_anti_affinity_matrix)
        solution_list.append(one_soln)
    return solution_list

In [19]:
def find_reources_left(new_individual,df_machines,df_applications,df_affinity_matrix,df_anti_affinity_matrix):
    df_remaining = df_machines.copy()
    for m in range(df_machines.shape[0]):
        for a in range(df_applications.shape[0]):
            for col in ['CPU','NW','IO','Memory']:
                df_remaining.loc[m,col] = df_remaining.loc[m,col] - new_individual.loc[a,m]*df_applications.loc[a,col]
    return df_remaining

In [20]:
def correct_the_individual(new_individual,df_machines,df_applications,df_affinity_matrix,df_anti_affinity_matrix):
#     remove all overflowing applications
    for m in range(df_machines.shape[0]):
        m_capacity = dict()
        m_filled = dict()
        yes = 0
        for col in ['CPU','NW','IO','Memory']:
            m_capacity[col] = df_machines.loc[m,col]
            m_filled[col] = 0
        for a in range(df_applications.shape[0]):
            if(yes == 1):
                new_individual.loc[a,m] = 0;
            else:
                number_of_instances = new_individual.loc[a,m]
                for col in ['CPU','NW','IO','Memory']:
                    m_filled[col] = m_filled[col] + df_applications.loc[a,col] * number_of_instances
                for col in ['CPU','NW','IO','Memory']:
                    if(m_filled[col] > m_capacity[col]):
                        yes = 1

#     fill all the remaining instances of applications in ffa
    df_remaining = find_reources_left(new_individual,df_machines,df_applications,df_affinity_matrix,df_anti_affinity_matrix)
    for a in range(df_applications.shape[0]):
        number_of_instances_required = df_applications.loc[a,'Instances']
        number_of_instances_scheduled = new_individual.loc[a,:].sum()
        number_of_instances_remaining = number_of_instances_required - number_of_instances_scheduled;
        for num in range(int(number_of_instances_remaining)):
            for m in range(df_machines.shape[0]):
                t_yes = 0
                for col in ['CPU','NW','IO','Memory']:
                    if(df_remaining.loc[m,col] < df_applications.loc[a,col]):
                        t_yes = 1;
                if(t_yes == 0):
                    new_individual.loc[a,m] = new_individual.loc[a,m]+1;
                    for col in ['CPU','NW','IO','Memory']:
                        df_remaining.loc[m,col] = df_remaining.loc[m,col] - df_applications.loc[a,col]
                    break;
    return new_individual

In [21]:
def generate_new_individuals_and_check(population,df_machines,df_applications,df_affinity_matrix,df_anti_affinity_matrix):
    new_individuals = []
    for i in range(NUM_NEW_POPULATION):
        random_numbers = random.sample(range(POPULATION_SIZE), 2)
        row = random.randint(0,df_applications.shape[0] - 1)
        col = random.randint(0,df_machines.shape[0] - 1)
        new_individual = population[random_numbers[0]].copy()
        
#         set rows below row as rn1
        new_individual.loc[row+1:,:] = population[random_numbers[1]].loc[row+1:,:]
        for i in range(col,df_machines.shape[0]):
            new_individual.loc[row,i]=population[random_numbers[1]].loc[row,i]
        
#         try to correct the individual
        new_individual = correct_the_individual(new_individual,df_machines,df_applications,df_affinity_matrix,df_anti_affinity_matrix)
        
#         check the individual
        instances_scheduled = new_individual.sum().sum()
        instances_required = df_applications['Instances'].sum()
        if(instances_scheduled == instances_required):
            new_individuals.append(new_individual)
    return new_individuals

In [22]:
def find_score(population,df_machines,df_applications,df_affinity_matrix,df_anti_affinity_matrix):
    answer_list = []
    for i in range(len(population)):
        candidate = population[i]
        temp_p = calculate_total_power_cost(candidate,df_applications,df_machines)
        temp_a = calculate_total_affinity_cost(candidate,df_affinity_matrix)
        temp_c = calculate_total_system_cost(temp_p,temp_a)
        answer_list.append(temp_c)
    return answer_list

In [23]:
def mutation(new_individual,df_machines,df_applications,df_affinity_matrix,df_anti_affinity_matrix):
    random_decimal = random.uniform(0, 1)
    if(random_decimal > MUTATION_PROB):
        for i in range(100):
            machines = random.sample(range(df_machines.shape[0]), 2)
            applications = random.sample(range(df_applications.shape[0]), 2)
            val1= new_individual.loc[applications[0],machines[0]]
            val2= new_individual.loc[applications[1],machines[1]]
            if(val1==0 or val2==0):
                continue;
            else:
#                 find resources left with m1
                df_remaining = find_reources_left(new_individual,df_machines,df_applications,df_affinity_matrix,df_anti_affinity_matrix)
#                 find resources left with m2
                t_yes=0
                for col in ['CPU','NW','IO','Memory']:
                    if(df_remaining.loc[machines[0],col]+df_applications.loc[applications[0],col]-df_applications.loc[applications[1],col] <0):
                        t_yes=1
                        break
                    elif(df_remaining.loc[machines[1],col]+df_applications.loc[applications[1],col]-df_applications.loc[applications[0],col] <0):
                        t_yes=1
                        break
                if(t_yes==0):
#                     change the numbers
                    new_individual.loc[applications[1],machines[0]]=new_individual.loc[applications[1],machines[0]]+1
                    new_individual.loc[applications[1],machines[1]]=new_individual.loc[applications[1],machines[1]]-1
                    new_individual.loc[applications[0],machines[0]]=new_individual.loc[applications[0],machines[0]]-1
                    new_individual.loc[applications[0],machines[1]]=new_individual.loc[applications[0],machines[1]]+1
                    break
        return new_individual
    else:
        return new_individual

In [24]:
def Genetic_Algorithm(df_machines,df_applications,df_affinity_matrix,df_anti_affinity_matrix):
#     population initialization
    population = generate_population(df_machines,df_applications,df_affinity_matrix,df_anti_affinity_matrix)
    population[0] = PAS(df_machines,df_applications,df_affinity_matrix,df_anti_affinity_matrix)
    population[1] = AAS(df_machines,df_applications,df_affinity_matrix,df_anti_affinity_matrix)
    population[2] = Proposed_Approach(df_machines,df_applications,df_affinity_matrix,df_anti_affinity_matrix)
    score_list = find_score(population,df_machines,df_applications,df_affinity_matrix,df_anti_affinity_matrix)
    avg_cost_over_iters = []
    avg_cost_over_iters.append(sum(score_list)/len(score_list))
    print('Population Initialized')
    for i in range(NUMBER_OF_GA_ITERS):
        new_individuals = generate_new_individuals_and_check(population,df_machines,df_applications,df_affinity_matrix,df_anti_affinity_matrix)
        for j in range(len(new_individuals)):
            mutated_individual = mutation(new_individuals[j],df_machines,df_applications,df_affinity_matrix,df_anti_affinity_matrix)
#             update the population with best individuals
            temp_p = calculate_total_power_cost(mutated_individual,df_applications,df_machines)
            temp_a = calculate_total_affinity_cost(mutated_individual,df_affinity_matrix)
            temp_c = calculate_total_system_cost(temp_p,temp_a)
            max_value_in_list = max(score_list)
            if(max_value_in_list > temp_c):
#                 find the index and replace the values
                index_me = score_list.index(max_value_in_list);
                population[index_me] = mutated_individual
                score_list[index_me] = temp_c
        avg_cost_over_iters.append(sum(score_list)/len(score_list))
        print('Iteration ',i+1, ': ', min(score_list))
    index_best = score_list.index(min(score_list))
    df_allocation_matrix = population[index_best]
    plt.plot(avg_cost_over_iters)
    plt.show()
    return df_allocation_matrix

# Simulated Annealing

In [25]:
def generate_candidate_helper(curr_df,step_size,df_machines,df_applications,df_affinity_matrix,df_anti_affinity_matrix):
    new_df = curr_df.copy()
    random_application = random.randint(0, df_applications.shape[0] - 1)
    random_machine = random.randint(0, df_machines.shape[0] - 1)
    while(df_anti_affinity_matrix.loc[random_application,random_machine] == 1):
        random_application = random.randint(0, df_applications.shape[0]-1)
        random_machine = random.randint(0,df_machines.shape[0]-1)
#     define the limits of change in ijth position
    value_original = curr_df.loc[random_application ,random_machine]
    max_changed_value = value_original + step_size
    min_changed_value = max(0,value_original - step_size)
    max_incr = step_size
    for col in ['CPU', 'IO', 'NW', 'Memory']:
        resources_used = (curr_df[random_machine] * df_applications[col]).sum()
        resources_available = df_machines.loc[random_machine,col]
        resource_required_by_one_application = df_applications.loc[random_application,col]
        max_incr = min(max_incr, int((resources_available - resources_used) / resource_required_by_one_application))
    
    max_changed_value = min(max_changed_value, value_original + max_incr)
    value_changed = random.randint(min_changed_value, max_changed_value)
    new_df.loc[random_application,random_machine] = value_changed
    random_change = value_changed - value_original

    if(random_change > 0):
#         subtract the allocations from the row - till no more required
        for index_machine in range(df_machines.shape[0]):
            entry = new_df.loc[random_application,index_machine]
            if(random_change >= entry):
                new_df.loc[random_application,index_machine] = 0
                random_change = random_change - entry
            else:
                new_df.loc[random_application,index_machine] = entry - random_change
                break
    elif(random_change < 0):
        num_to_add = -random_change
        for index_machine in range(df_machines.shape[0]):
            max_number_to_add_possible = df_applications.shape[0]
            for col in ['CPU', 'IO', 'NW', 'Memory']:
                resources_used = (new_df[index_machine] * df_applications[col]).sum()
                resources_available = df_machines.loc[index_machine,col]
                resource_required_by_one_application = df_applications.loc[random_application,col]
                max_number_to_add_possible = min(max_number_to_add_possible, int((resources_available - resources_used) / resource_required_by_one_application))
            if(max_number_to_add_possible <= num_to_add):
                new_df.loc[random_application,index_machine] = new_df.loc[random_application,index_machine] + max_number_to_add_possible
                num_to_add = num_to_add - max_number_to_add_possible
            else:
                new_df.loc[random_application,index_machine] = new_df.loc[random_application,index_machine] + num_to_add
                break
    return new_df

In [26]:
def generate_candidate(curr_df,step_size,df_machines,df_applications,df_affinity_matrix,df_anti_affinity_matrix):
#     how to generate a new feasible matrix which is step size away from curr_df
#     we are not using interfernce matrix and affinity matrix for this generation
#     we select a random ij position till ij is not antiaffine
    curr_number_of_instances = curr_df.sum().sum()
    new_df = generate_candidate_helper(curr_df,step_size,df_machines,df_applications,df_affinity_matrix,df_anti_affinity_matrix)
    for i in range(NUMBER_OF_NEW_CAND_ITERS):
        new_number_of_instances = new_df.sum().sum()
        if(new_number_of_instances < curr_number_of_instances):
            new_df = generate_candidate_helper(curr_df,step_size,df_machines,df_applications,df_affinity_matrix,df_anti_affinity_matrix)
        else:
            break
    return new_df

In [27]:
def Simulated_Annealing(df_machines,df_applications,df_affinity_matrix,df_anti_affinity_matrix):
    best_df_allocation_matrix = generate_suitable_candidate(df_machines,df_applications,df_affinity_matrix,df_anti_affinity_matrix)
    best_total_power_cost = calculate_total_power_cost(best_df_allocation_matrix,df_applications,df_machines)
    best_total_affinity_cost = calculate_total_affinity_cost(best_df_allocation_matrix,df_affinity_matrix)
    best_total_cost_of_system = calculate_total_system_cost(best_total_power_cost,best_total_affinity_cost)
    curr_df_allocation_matrix = best_df_allocation_matrix
    curr_total_cost_of_system  = best_total_cost_of_system
    list_costs = [curr_total_cost_of_system]
    for i in range(NUMBER_OF_SA_ITERS):
        candidate_df = generate_candidate(curr_df_allocation_matrix,STEP_SIZE,df_machines,df_applications,df_affinity_matrix,df_anti_affinity_matrix)
        candidate_total_power_cost = calculate_total_power_cost(candidate_df,df_applications,df_machines)
        candidate_total_affinity_cost = calculate_total_affinity_cost(candidate_df,df_affinity_matrix)
        candidate_total_cost_of_system = calculate_total_system_cost(candidate_total_power_cost,candidate_total_affinity_cost)
#         list_costs.append(candidate_total_cost_of_system)
        if(candidate_total_cost_of_system < curr_total_cost_of_system):
            best_df_allocation_matrix = candidate_df
            best_total_cost_of_system = candidate_total_cost_of_system
        diff = candidate_total_cost_of_system - curr_total_cost_of_system
        t = TEMP/float(i+1)
        metropolis = math.exp(-diff / t)
        if diff < 0 or np.random.rand() < metropolis:
            curr_df_allocation_matrix, curr_total_cost_of_system = candidate_df, candidate_total_cost_of_system
        list_costs.append(best_total_cost_of_system)
#         print('Iteration ',i+1, ': ', best_total_cost_of_system)
    
    plt.plot(list_costs)
    plt.show()
    return best_df_allocation_matrix

# Greedy Heuristics

In [28]:
def check_if_possible(df_machines, df_applications, df_allocation_matrix, i, j):
    for col in ['CPU', 'IO', 'NW', 'Memory']:
        if ((df_allocation_matrix[j] * df_applications[col]).sum() + df_applications[col][i] > df_machines[col][j]):
            return False
    return True

In [29]:
def PAS(df_machines, df_applications, df_affinity_matrix, df_anti_affinity_matrix, knee = 0.45):
    df_applications_copy = df_applications.copy().sort_values(by = ['CPU', 'IO', 'NW', 'Memory'], ascending = [False, False, False, False])
    df_machines_copy = df_machines.copy()
    df_allocation_matrix = pd.DataFrame(np.zeros((df_applications.shape[0], df_machines.shape[0])))
    for i, application in df_applications_copy.iterrows():
        for k in range(int(application['Instances'])):
            df_machines_copy['utilization'] = calculate_utilization(df_allocation_matrix, df_applications, df_machines)
            df_machines_copy['utilization'] = [1 if df_machines_copy['utilization'][j] > knee else df_machines_copy['utilization'][j] for j in range(df_machines.shape[0])]
            df_machines_copy['affinity'] = df_affinity_matrix.loc[application['Index']]
            df_machines_copy['power'] = (df_machines_copy['P_max'] - df_machines_copy['P_idle'])
            df_machines_copy_2 = df_machines_copy.sort_values(by = ['utilization', 'affinity', 'power'], ascending = [True, False, True])
            for j, machine in df_machines_copy_2.iterrows():
                if (df_anti_affinity_matrix.loc[application['Index'], machine['Index']] == 1):
                    continue
                elif (application['CPU'] > machine['CPU'] or application['IO'] > machine['IO'] or application['NW'] > machine['NW'] or application['Memory'] > machine['Memory']):
                    continue
                elif (not check_if_possible(df_machines, df_applications, df_allocation_matrix, application['Index'], machine['Index'])):
                    continue
                else:
                    df_allocation_matrix.loc[application['Index'], machine['Index']] += 1
                    break
    return df_allocation_matrix

In [30]:
def AAS(df_machines, df_applications, df_affinity_matrix, df_anti_affinity_matrix):
    df_applications_copy = df_applications.copy().sort_values(by = ['CPU', 'IO', 'NW', 'Memory'], ascending = [False, False, False, False])
    df_machines_copy = df_machines.copy()
    df_allocation_matrix = pd.DataFrame(np.zeros((df_applications.shape[0], df_machines.shape[0])))
    for i, application in df_applications_copy.iterrows():
        for k in range(int(application['Instances'])):
            df_machines_copy['utilization'] = calculate_utilization(df_allocation_matrix, df_applications, df_machines)
            df_machines_copy['affinity'] = df_affinity_matrix.loc[application['Index']]
            df_machines_copy['power'] = (df_machines_copy['P_max'] - df_machines_copy['P_idle'])
            df_machines_copy_2 = df_machines_copy.sort_values(by = ['affinity', 'utilization', 'power'], ascending = [False, True, True])
            for j, machine in df_machines_copy_2.iterrows():
                if (df_anti_affinity_matrix.loc[application['Index'], machine['Index']] == 1):
                    continue
                elif (application['CPU'] > machine['CPU'] or application['IO'] > machine['IO'] or application['NW'] > machine['NW'] or application['Memory'] > machine['Memory']):
                    continue
                elif (not check_if_possible(df_machines, df_applications, df_allocation_matrix, application['Index'], machine['Index'])):
                    continue
                else:
                    df_allocation_matrix.loc[application['Index'], machine['Index']] += 1
                    break
    return df_allocation_matrix

In [31]:
def Proposed_Approach(df_machines, df_applications, df_affinity_matrix, df_anti_affinity_matrix):
    df_applications_copy = df_applications.copy().sort_values(by = ['CPU', 'IO', 'NW', 'Memory'], ascending = [False, False, False, False])
    df_machines_copy = df_machines.copy()
    df_allocation_matrix = pd.DataFrame(np.zeros((df_applications.shape[0], df_machines.shape[0])))
    for i, application in df_applications_copy.iterrows():
        for k in range(int(application['Instances'])):
            df_machines_copy['utilization'] = calculate_utilization(df_allocation_matrix, df_applications, df_machines)
            df_machines_copy['affinity'] = df_affinity_matrix.loc[application['Index']]
            df_machines_copy['power'] = (df_machines_copy['P_max'] - df_machines_copy['P_idle'])
            df_machines_copy_2 = df_machines_copy.sort_values(by = ['utilization', 'affinity', 'power'], ascending = [True, False, True])
            df_machines_copy_3 = df_machines_copy.sort_values(by = ['affinity', 'utilization', 'power'], ascending = [False, True, True])
            
            sol_1 = []
            sol_2 = []
            for j, machine in df_machines_copy_2.iterrows():
                if (df_anti_affinity_matrix.loc[application['Index'], machine['Index']] == 1):
                    continue
                elif (application['CPU'] > machine['CPU'] or application['IO'] > machine['IO'] or application['NW'] > machine['NW'] or application['Memory'] > machine['Memory']):
                    continue
                elif (not check_if_possible(df_machines, df_applications, df_allocation_matrix, application['Index'], machine['Index'])):
                    continue
                else:
                    df_allocation_matrix.loc[application['Index'], machine['Index']] += 1
                    power_cost_temp = calculate_total_power_cost(df_allocation_matrix, df_applications, df_machines)
                    affinity_cost_temp = calculate_total_affinity_cost(df_allocation_matrix, df_affinity_matrix)
                    total_cost_temp = calculate_total_system_cost(power_cost_temp, affinity_cost_temp)
                    sol_1 = [application['Index'], machine['Index'], total_cost_temp]
                    df_allocation_matrix.loc[application['Index'], machine['Index']] -= 1
                    break
            
            for j, machine in df_machines_copy_3.iterrows():
                if (df_anti_affinity_matrix.loc[application['Index'], machine['Index']] == 1):
                    continue
                elif (application['CPU'] > machine['CPU'] or application['IO'] > machine['IO'] or application['NW'] > machine['NW'] or application['Memory'] > machine['Memory']):
                    continue
                elif (not check_if_possible(df_machines, df_applications, df_allocation_matrix, application['Index'], machine['Index'])):
                    continue
                else:
                    df_allocation_matrix.loc[application['Index'], machine['Index']] += 1
                    power_cost_temp = calculate_total_power_cost(df_allocation_matrix, df_applications, df_machines)
                    affinity_cost_temp = calculate_total_affinity_cost(df_allocation_matrix, df_affinity_matrix)
                    total_cost_temp = calculate_total_system_cost(power_cost_temp, affinity_cost_temp)
                    sol_2 = [application['Index'], machine['Index'], total_cost_temp]
                    df_allocation_matrix.loc[application['Index'], machine['Index']] -= 1
                    break
            
            if sol_1[2] > sol_2[2]:
                df_allocation_matrix.loc[sol_2[0], sol_2[1]] += 1
            else:
                df_allocation_matrix.loc[sol_1[0], sol_1[1]] += 1
    return df_allocation_matrix

# Grouping and Scheduling

In [32]:
def filter_out_machines(df_machines, df_applications, df_allocation_matrix, asc):
    CPU_th_2 = df_machines['CPU'].max() * 0.7
    if not asc[0]:
        CPU_th_2 = 0
    IO_th_2 = df_machines['IO'].max() * 0.7
    if not asc[1]:
        IO_th_2 = 0
    NW_th_2 = df_machines['NW'].max() * 0.7
    if not asc[2]:
        NW_th_2 = 0
    MEM_th_2 = df_machines['Memory'].max() * 0.7
    if not asc[3]:
        MEM_th_2 = 0
    
    for col in ['CPU', 'IO', 'NW', 'Memory']:
        for j in range(df_machines.shape[0]):
            df_machines.loc[j,col] = df_machines.loc[j,col] - ((df_allocation_matrix[j] * df_applications[col]).sum())
    
    return df_machines[(df_machines['CPU'] > CPU_th_2) & (df_machines['IO'] > IO_th_2) & (df_machines['NW'] > NW_th_2) & (df_machines['Memory'] > MEM_th_2)]

In [33]:
def Group_Sched(df_machines, df_applications, df_affinity_matrix, df_anti_affinity_matrix):
    df_allocation_matrix = pd.DataFrame(np.zeros((df_applications.shape[0], df_machines.shape[0])))
    df_applications_copy = df_applications.copy().sort_values(by = ['CPU', 'IO', 'NW', 'Memory'], ascending = [False, False, False, False])
    CPU_th = df_applications_copy['CPU'].max() * 0.7
    IO_th = df_applications_copy['IO'].max() * 0.7
    NW_th = df_applications_copy['NW'].max() * 0.7
    MEM_th = df_applications_copy['Memory'].max() * 0.7
    for i, application in df_applications_copy.iterrows():
        for k in range(int(application['Instances'])):
            df_machines_copy = df_machines.copy()
            df_machines_copy['utilization'] = calculate_utilization(df_allocation_matrix, df_applications, df_machines)
            df_machines_copy['affinity'] = df_affinity_matrix.loc[application['Index']]
            df_machines_copy['power'] = (df_machines_copy['P_max'] - df_machines_copy['P_idle']) * (df_machines_copy['utilization'] ** 3)
            asc = [application['CPU'] > CPU_th, application['IO'] > IO_th, application['NW'] > NW_th, application['Memory'] > MEM_th]
            df_machines_copy_2 = filter_out_machines(df_machines_copy, df_applications, df_allocation_matrix, asc)
            df_machines_copy_2 = df_machines_copy_2.sort_values(by = ['power', 'affinity'], ascending = [True, False])
            sched = False
            for j, machine in df_machines_copy_2.iterrows():
                if (df_anti_affinity_matrix.loc[application['Index'], machine['Index']] == 1):
                    continue
                elif (application['CPU'] > machine['CPU'] or application['IO'] > machine['IO'] or application['NW'] > machine['NW'] or application['Memory'] > machine['Memory']):
                    continue
                elif (not check_if_possible(df_machines, df_applications, df_allocation_matrix, application['Index'], machine['Index'])):
                    continue
                else:
                    df_allocation_matrix.loc[application['Index'], machine['Index']] += 1
                    sched = True
                    break
            
            if not sched:
                df_machines_copy_2 = df_machines_copy.sort_values(by = ['power', 'affinity'], ascending = [True, False])
                for j, machine in df_machines_copy_2.iterrows():
                    if (df_anti_affinity_matrix.loc[application['Index'], machine['Index']] == 1):
                        continue
                    elif (application['CPU'] > machine['CPU'] or application['IO'] > machine['IO'] or application['NW'] > machine['NW'] or application['Memory'] > machine['Memory']):
                        continue
                    elif (not check_if_possible(df_machines, df_applications, df_allocation_matrix, application['Index'], machine['Index'])):
                        continue
                    else:
                        df_allocation_matrix.loc[application['Index'], machine['Index']] += 1
                        break
    
    return df_allocation_matrix

# Simulation

## Impact of Number of Machines (Random Dataset)
- Alpha = 4
- Applications: 18
- Machines : [20, 30, 40, 50, 60]

In [34]:
# for algo in [Group_Sched, PAS, AAS, Proposed_Approach]:
#     print(algo.__name__)
#     pc = []
#     ac = []
#     tc = []
#     aasr = []
#     au = []
#     for num_applications in [18]:
#         for num_machines in [20, 30, 40, 50, 60]:
#             number_of_machines_temp = num_machines
#             number_of_applications_temp = num_applications
#             df_machines_temp = df_machines.head(number_of_machines_temp)
#             df_applications_temp = df_applications.head(number_of_applications_temp)
#             df_affinity_matrix_temp = df_affinity_matrix.loc[0:number_of_applications_temp-1,0:number_of_machines_temp-1]
#             df_user_affinity_matrix_temp = df_user_affinity_matrix.loc[0:number_of_applications_temp-1,0:number_of_machines_temp-1]
#             df_anti_affinity_matrix_temp = df_anti_affinity_matrix.loc[0:number_of_applications_temp-1,0:number_of_machines_temp-1]

#             df_allocation_matrix = algo(df_machines_temp, df_applications_temp, df_affinity_matrix_temp, df_anti_affinity_matrix_temp)
#             if df_allocation_matrix.sum().sum() != df_applications_temp['Instances'].sum():
#                 print("False!")
#             power_cost = calculate_total_power_cost(df_allocation_matrix, df_applications_temp, df_machines_temp)
#             affinity_cost = calculate_total_affinity_cost(df_allocation_matrix, df_affinity_matrix_temp)
#             tc.append(calculate_total_system_cost(power_cost, affinity_cost))
#             pc.append(power_cost)
#             ac.append(affinity_cost)
#             aasr.append(calculate_average_affinity_satisfaction_ratio(df_applications_temp, df_machines_temp, df_allocation_matrix, df_user_affinity_matrix_temp))
#             au.append(calculate_average_utilization(df_allocation_matrix, df_applications_temp, df_machines_temp))
#             print("Done!")
#     print(pc)
#     print(ac)
#     print(tc)
#     print(aasr)
#     print(au)

## Impact of Number of Machines (Benchmark Dataset)

- Alpha = 4
- Applications: 175
- Machines : [150, 175, 200, 225, 250]

In [35]:
# for algo in [Group_Sched, PAS, AAS, Proposed_Approach]:
#     print(algo.__name__)
#     pc = []
#     ac = []
#     tc = []
#     aasr = []
#     au = []
#     for num_applications in [175]:
#         for num_machines in [150, 175, 200, 225, 250]:
#             number_of_machines_temp = num_machines
#             number_of_applications_temp = num_applications
#             df_machines_temp = df_machines.head(number_of_machines_temp)
#             df_applications_temp = df_applications.head(number_of_applications_temp)
#             df_affinity_matrix_temp = df_affinity_matrix.loc[0:number_of_applications_temp-1,0:number_of_machines_temp-1]
#             df_user_affinity_matrix_temp = df_user_affinity_matrix.loc[0:number_of_applications_temp-1,0:number_of_machines_temp-1]
#             df_anti_affinity_matrix_temp = df_anti_affinity_matrix.loc[0:number_of_applications_temp-1,0:number_of_machines_temp-1]

#             df_allocation_matrix = algo(df_machines_temp, df_applications_temp, df_affinity_matrix_temp, df_anti_affinity_matrix_temp)
#             if df_allocation_matrix.sum().sum() != df_applications_temp['Instances'].sum():
#                 print("False!")
#             power_cost = calculate_total_power_cost(df_allocation_matrix, df_applications_temp, df_machines_temp)
#             affinity_cost = calculate_total_affinity_cost(df_allocation_matrix, df_affinity_matrix_temp)
#             tc.append(calculate_total_system_cost(power_cost, affinity_cost))
#             pc.append(power_cost)
#             ac.append(affinity_cost)
#             aasr.append(calculate_average_affinity_satisfaction_ratio(df_applications_temp, df_machines_temp, df_allocation_matrix, df_user_affinity_matrix_temp))
#             au.append(calculate_average_utilization(df_allocation_matrix, df_applications_temp, df_machines_temp))
#             print("Done!")
#     print(pc)
#     print(ac)
#     print(tc)
#     print(aasr)
#     print(au)

## Impact of Number of Applications (Random Dataset)

- Alpha = 4
- Applications: [30, 40, 50, 60, 70]
- Machines: 60

In [36]:
# for algo in [Group_Sched, PAS, AAS, Proposed_Approach]:
#     print(algo.__name__)
#     pc = []
#     ac = []
#     tc = []
#     aasr = []
#     au = []
#     for num_applications in [30, 40, 50, 60, 70]:
#         for num_machines in [60]:
#             number_of_machines_temp = num_machines
#             number_of_applications_temp = num_applications
#             df_machines_temp = df_machines.head(number_of_machines_temp)
#             df_applications_temp = df_applications.head(number_of_applications_temp)
#             df_affinity_matrix_temp = df_affinity_matrix.loc[0:number_of_applications_temp-1,0:number_of_machines_temp-1]
#             df_user_affinity_matrix_temp = df_user_affinity_matrix.loc[0:number_of_applications_temp-1,0:number_of_machines_temp-1]
#             df_anti_affinity_matrix_temp = df_anti_affinity_matrix.loc[0:number_of_applications_temp-1,0:number_of_machines_temp-1]

#             df_allocation_matrix = algo(df_machines_temp, df_applications_temp, df_affinity_matrix_temp, df_anti_affinity_matrix_temp)
#             if df_allocation_matrix.sum().sum() != df_applications_temp['Instances'].sum():
#                 print("False!")
#             power_cost = calculate_total_power_cost(df_allocation_matrix, df_applications_temp, df_machines_temp)
#             affinity_cost = calculate_total_affinity_cost(df_allocation_matrix, df_affinity_matrix_temp)
#             tc.append(calculate_total_system_cost(power_cost, affinity_cost))
#             pc.append(power_cost)
#             ac.append(affinity_cost)
#             aasr.append(calculate_average_affinity_satisfaction_ratio(df_applications_temp, df_machines_temp, df_allocation_matrix, df_user_affinity_matrix_temp))
#             au.append(calculate_average_utilization(df_allocation_matrix, df_applications_temp, df_machines_temp))
#             print("Done!")
#     print(pc)
#     print(ac)
#     print(tc)
#     print(aasr)
#     print(au)

## Impact of Number of Applications (Benchmark Dataset)

- Alpha = 4
- Applications: [120 ,140, 160, 180, 200]
- Machines: 250

In [37]:
# for algo in [Group_Sched, PAS, AAS, Proposed_Approach]:
#     print(algo.__name__)
#     pc = []
#     ac = []
#     tc = []
#     aasr = []
#     au = []
#     for num_applications in [120 ,140, 160, 180, 200]:
#         for num_machines in [250]:
#             number_of_machines_temp = num_machines
#             number_of_applications_temp = num_applications
#             df_machines_temp = df_machines.head(number_of_machines_temp)
#             df_applications_temp = df_applications.head(number_of_applications_temp)
#             df_affinity_matrix_temp = df_affinity_matrix.loc[0:number_of_applications_temp-1,0:number_of_machines_temp-1]
#             df_user_affinity_matrix_temp = df_user_affinity_matrix.loc[0:number_of_applications_temp-1,0:number_of_machines_temp-1]
#             df_anti_affinity_matrix_temp = df_anti_affinity_matrix.loc[0:number_of_applications_temp-1,0:number_of_machines_temp-1]

#             df_allocation_matrix = algo(df_machines_temp, df_applications_temp, df_affinity_matrix_temp, df_anti_affinity_matrix_temp)
#             if df_allocation_matrix.sum().sum() != df_applications_temp['Instances'].sum():
#                 print("False!")
#             power_cost = calculate_total_power_cost(df_allocation_matrix, df_applications_temp, df_machines_temp)
#             affinity_cost = calculate_total_affinity_cost(df_allocation_matrix, df_affinity_matrix_temp)
#             tc.append(calculate_total_system_cost(power_cost, affinity_cost))
#             pc.append(power_cost)
#             ac.append(affinity_cost)
#             aasr.append(calculate_average_affinity_satisfaction_ratio(df_applications_temp, df_machines_temp, df_allocation_matrix, df_user_affinity_matrix_temp))
#             au.append(calculate_average_utilization(df_allocation_matrix, df_applications_temp, df_machines_temp))
#             print("Done!")
#     print(pc)
#     print(ac)
#     print(tc)
#     print(aasr)
#     print(au)

## Impact of ALPHA (Benchmark Dataset)

- Alpha = [0.5, 1, 5, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
- Applications: 200
- Machines: 250

In [38]:
# for algo in [Group_Sched, PAS, AAS, Proposed_Approach]:
#     print(algo.__name__)
#     tc = []
#     for ALPHA in [0.5, 1, 5, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]:
#         number_of_machines_temp = df_machines.shape[0]
#         number_of_applications_temp = df_applications.shape[0]
#         df_machines_temp = df_machines.head(number_of_machines_temp)
#         df_applications_temp = df_applications.head(number_of_applications_temp)
#         df_affinity_matrix_temp = df_affinity_matrix.loc[0:number_of_applications_temp-1,0:number_of_machines_temp-1]
#         df_user_affinity_matrix_temp = df_user_affinity_matrix.loc[0:number_of_applications_temp-1,0:number_of_machines_temp-1]
#         df_anti_affinity_matrix_temp = df_anti_affinity_matrix.loc[0:number_of_applications_temp-1,0:number_of_machines_temp-1]

#         df_allocation_matrix = algo(df_machines_temp, df_applications_temp, df_affinity_matrix_temp, df_anti_affinity_matrix_temp)
#         if df_allocation_matrix.sum().sum() != df_applications_temp['Instances'].sum():
#             print("False!")
#         power_cost = calculate_total_power_cost(df_allocation_matrix, df_applications_temp, df_machines_temp)
#         affinity_cost = calculate_total_affinity_cost(df_allocation_matrix, df_affinity_matrix_temp)
#         tc.append(calculate_total_system_cost(power_cost, affinity_cost))
#         print("Done!")
#     print(tc)

## Impact of Anti-Affinity Percetage (Benchmark Dataset)

- Alpha = 4
- Applications: 200
- Machines: 250
- Anti-Affinity Percentage = [10%, 20%, 30%, 40%, 50%]

In [39]:
# for algo in [Group_Sched, PAS, AAS, Proposed_Approach]:
#     print(algo.__name__)
#     pc = []
#     ac = []
#     tc = []
#     aasr = []
#     au = []
#     for percent in [10, 20, 30, 40, 50]:
#         df_anti_affinity_matrix = pd.read_csv('../Datasets/Anti-Affinity Matrices/Anti_Affinity_Matrix_' + str(percent) + '.csv')
#         df_anti_affinity_matrix = df_anti_affinity_matrix.loc[:, ~df_anti_affinity_matrix.columns.str.contains('^Unnamed')]
#         df_anti_affinity_matrix.columns = pd.to_numeric(df_anti_affinity_matrix.columns, errors='coerce')

#         number_of_machines_temp = df_machines.shape[0]
#         number_of_applications_temp = df_applications.shape[0]
#         df_machines_temp = df_machines.head(number_of_machines_temp)
#         df_applications_temp = df_applications.head(number_of_applications_temp)
#         df_affinity_matrix_temp = df_affinity_matrix.loc[0:number_of_applications_temp-1,0:number_of_machines_temp-1]
#         df_user_affinity_matrix_temp = df_user_affinity_matrix.loc[0:number_of_applications_temp-1,0:number_of_machines_temp-1]
#         df_anti_affinity_matrix_temp = df_anti_affinity_matrix.loc[0:number_of_applications_temp-1,0:number_of_machines_temp-1]

#         df_allocation_matrix = algo(df_machines_temp, df_applications_temp, df_affinity_matrix_temp, df_anti_affinity_matrix_temp)
#         if df_allocation_matrix.sum().sum() != df_applications_temp['Instances'].sum():
#             print("False!")
#         power_cost = calculate_total_power_cost(df_allocation_matrix, df_applications_temp, df_machines_temp)
#         affinity_cost = calculate_total_affinity_cost(df_allocation_matrix, df_affinity_matrix_temp)
#         tc.append(calculate_total_system_cost(power_cost, affinity_cost))
#         pc.append(power_cost)
#         ac.append(affinity_cost)
#         aasr.append(calculate_average_affinity_satisfaction_ratio(df_applications_temp, df_machines_temp, df_allocation_matrix, df_user_affinity_matrix_temp))
#         au.append(calculate_average_utilization(df_allocation_matrix, df_applications_temp, df_machines_temp))
#         print("Done!")
#     print(pc)
#     print(ac)
#     print(tc)
#     print(aasr)
#     print(au)